In [ ]:
import os
import cv2
import torch
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
from rembg import remove  

In [7]:
input_dir = "D:\MED_LEAF_ID-1\dataset\Medicinal Leaf dataset"
output_dir = "D:/MED_LEAF_ID/preprocessed_glcm"
os.makedirs(output_dir, exist_ok=True)

In [8]:
allowed_extensions = {".jpg", ".jpeg", ".png", ".bmp"}

In [ ]:
def preprocess_image(image):
   
    pil_image = Image.fromarray(image)
    output_pil = remove(pil_image)
    output_np = np.array(output_pil)
    
    if output_np.shape[2] == 4:  
        output_np = cv2.cvtColor(output_np, cv2.COLOR_BGRA2BGR)  
    gray = cv2.cvtColor(output_np, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    edges = cv2.Sobel(blurred, cv2.CV_64F, 1, 1, ksize=3)
    edges = cv2.convertScaleAbs(edges)
    combined = cv2.addWeighted(blurred, 0.7, edges, 0.3, 0)

    normalized = cv2.normalize(combined, None, 0, 255, cv2.NORM_MINMAX)

    _, mask = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    output_np[mask == 0] = (255, 255, 255)  

    return output_np

In [ ]:
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),  
    transforms.ColorJitter(brightness=0.1, contrast=0.1), 
    transforms.GaussianBlur(kernel_size=5), 
])

In [ ]:
for root, _, files in os.walk(input_dir):
    for filename in files:
        if os.path.splitext(filename)[1].lower() in allowed_extensions:  
            img_path = os.path.join(root, filename)

            # Read Image
            image = cv2.imread(img_path)

            if image is None:
                print(f" Skipping unreadable image: {img_path}")
                continue

            processed_img = preprocess_image(image)
            processed_pil = Image.fromarray(processed_img)
            augmented_pil = augmentations(processed_pil)
            augmented_np = np.array(augmented_pil)
            relative_path = os.path.relpath(img_path, input_dir)
            save_path = os.path.join(output_dir, relative_path)
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            cv2.imwrite(save_path, augmented_np)

            print(f"Saved: {save_path}")

print("GLCM Preprocessing Completed!")